# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 3 2022 11:10AM,238753,10,BioHeat-KitEuro,Bio-PRF,Released
1,Jun 3 2022 11:10AM,238752,10,BioHeat-KitUSA,Bio-PRF,Released
2,Jun 3 2022 11:09AM,238751,10,BPS1241,Beach Patient Assistance,Released
3,Jun 3 2022 11:05AM,238749,19,DEX0006931,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
4,Jun 3 2022 11:05AM,238749,19,DEX0006932,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jun 3 2022 11:05AM,238749,19,DEX0006933,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,Jun 3 2022 11:05AM,238749,19,DEX0006934,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
7,Jun 3 2022 11:05AM,238749,19,DEX0006935,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
8,Jun 3 2022 11:05AM,238749,19,DEX0006936,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
9,Jun 3 2022 11:05AM,238749,19,DEX0006937,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,238748,Released,1
39,238749,Released,10
40,238751,Released,1
41,238752,Released,1
42,238753,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238748,NaN,NaN,1.0
238749,NaN,NaN,10.0
238751,NaN,NaN,1.0
238752,NaN,NaN,1.0
238753,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238647,0.0,0.0,1.0
238649,15.0,9.0,0.0
238650,0.0,0.0,1.0
238654,0.0,0.0,1.0
238662,0.0,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238647,0,0,1
238649,15,9,0
238650,0,0,1
238654,0,0,1
238662,0,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238647,0,0,1
1,238649,15,9,0
2,238650,0,0,1
3,238654,0,0,1
4,238662,0,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238647,,,1
1,238649,15,9,
2,238650,,,1
3,238654,,,1
4,238662,,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 3 2022 11:10AM,238753,10,Bio-PRF
1,Jun 3 2022 11:10AM,238752,10,Bio-PRF
2,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance
3,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
13,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.
14,Jun 3 2022 10:42AM,238747,10,ISDIN Corporation
15,Jun 3 2022 10:22AM,238746,10,Eywa Pharma Inc.
22,Jun 3 2022 10:13AM,238745,16,Sartorius Bioprocess Solutions
23,Jun 3 2022 10:12AM,238744,10,"Amcyte Pharma, Inc."
25,Jun 3 2022 10:12AM,238720,18,"Blinc International, Inc. - PV"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 3 2022 11:10AM,238753,10,Bio-PRF,,,1
1,Jun 3 2022 11:10AM,238752,10,Bio-PRF,,,1
2,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,,,1
3,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,10
4,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.,,,1
5,Jun 3 2022 10:42AM,238747,10,ISDIN Corporation,,,1
6,Jun 3 2022 10:22AM,238746,10,Eywa Pharma Inc.,,,7
7,Jun 3 2022 10:13AM,238745,16,Sartorius Bioprocess Solutions,,,1
8,Jun 3 2022 10:12AM,238744,10,"Amcyte Pharma, Inc.",,,2
9,Jun 3 2022 10:12AM,238720,18,"Blinc International, Inc. - PV",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1,,
1,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1,,
2,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1,,
3,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10,,
4,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.,1,,
5,Jun 3 2022 10:42AM,238747,10,ISDIN Corporation,1,,
6,Jun 3 2022 10:22AM,238746,10,Eywa Pharma Inc.,7,,
7,Jun 3 2022 10:13AM,238745,16,Sartorius Bioprocess Solutions,1,,
8,Jun 3 2022 10:12AM,238744,10,"Amcyte Pharma, Inc.",2,,
9,Jun 3 2022 10:12AM,238720,18,"Blinc International, Inc. - PV",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1,,
1,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1,,
2,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1,,
3,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10,,
4,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1.0,NaN,NaN
1,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1.0,NaN,NaN
2,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1.0,NaN,NaN
3,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10.0,NaN,NaN
4,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 3 2022 11:10AM,238753,10,Bio-PRF,1.0,0.0,0.0
1,Jun 3 2022 11:10AM,238752,10,Bio-PRF,1.0,0.0,0.0
2,Jun 3 2022 11:09AM,238751,10,Beach Patient Assistance,1.0,0.0,0.0
3,Jun 3 2022 11:05AM,238749,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,10.0,0.0,0.0
4,Jun 3 2022 10:48AM,238748,16,Sartorius Stedim Filters Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4535661,105.0,31.0,19.0
12,716176,25.0,1.0,0.0
16,716147,3.0,0.0,0.0
18,477441,2.0,0.0,0.0
19,716117,12.0,1.0,0.0
20,716145,14.0,19.0,6.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4535661,105.0,31.0,19.0
1,12,716176,25.0,1.0,0.0
2,16,716147,3.0,0.0,0.0
3,18,477441,2.0,0.0,0.0
4,19,716117,12.0,1.0,0.0
5,20,716145,14.0,19.0,6.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,105.0,31.0,19.0
1,12,25.0,1.0,0.0
2,16,3.0,0.0,0.0
3,18,2.0,0.0,0.0
4,19,12.0,1.0,0.0
5,20,14.0,19.0,6.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,105.0
1,12,Released,25.0
2,16,Released,3.0
3,18,Released,2.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20
Status,,,,,,
Completed,19.0,0.0,0.0,0.0,0.0,6.0
Executing,31.0,1.0,0.0,0.0,1.0,19.0
Released,105.0,25.0,3.0,2.0,12.0,14.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20
0,Completed,19.0,0.0,0.0,0.0,0.0,6.0
1,Executing,31.0,1.0,0.0,0.0,1.0,19.0
2,Released,105.0,25.0,3.0,2.0,12.0,14.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20
0,Completed,19.0,0.0,0.0,0.0,0.0,6.0
1,Executing,31.0,1.0,0.0,0.0,1.0,19.0
2,Released,105.0,25.0,3.0,2.0,12.0,14.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()